In [2]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

/Users/daphnehansell/miniconda3/envs/policyengine/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
year =2024

household_size = baseline.calculate("household_size", map_to= "household", period=year)
snap = baseline.calculate("snap",  map_to= "household", period=year)

In [3]:
data = {
    "household_size": household_size,
    "snap": snap,
}

In [11]:
import numpy as np
import pandas as pd

# Single month keeps it simple
period = "2024-01"

snap = baseline.calculate("snap", period=period)               # SPM-unit level
unit_size = baseline.calculate("spm_unit_size", period=period) # size of that SPM unit
weights = baseline.calculate("household_weight",
                             map_to="spm_unit", period=period) # population weights

mask = (unit_size == 4) & np.isclose(snap, 973, atol=1)

print("Households getting maximum snap:  ", float((mask * weights).sum()))


Households getting maximum snap:   51343.10517691474


In [9]:
import numpy as np
import pandas as pd

# Single month keeps it simple
period = "2024-01"

snap = baseline.calculate("snap", period=period)               # SPM-unit level
unit_size = baseline.calculate("spm_unit_size", period=period) # size of that SPM unit

(unit_size == 4)[snap >= 1].sum()


2.11511349654758

In [7]:
snap[unit_size == 4].max()

1455.7

In [5]:
pd.Series(snap[unit_size == 4]).value_counts().head(10)


0.000000      4561
973.000000     184
95.000000        3
363.399963       2
705.400024       2
548.199951       2
744.700012       2
790.299988       2
573.400024       2
488.499969       2
Name: count, dtype: int64

In [9]:
import pandas as pd
import microdf as mdf
import numpy as np
import pandas as pd

# Single month keeps it simple
period = "2024-01"

snap = baseline.calculate("snap", period=period)               # SPM-unit level
unit_size = baseline.calculate("spm_unit_size", period=period) # size of that SPM unit
weights = baseline.calculate("household_weight",
                             map_to="spm_unit", period=period) # population weights

mask = (unit_size == 4) & np.isclose(snap, 750, atol=1)

print("4 Person Households that get Maximum SNAP Allotment :  ", float((mask * weights).sum()))

period = "2024-01"      # keep benefits in monthly dollars

# ── Pull variables at the SPM-unit level ──
snap       = baseline.calculate("snap",          period=period)
unit_size  = baseline.calculate("spm_unit_size", period=period)
weights    = baseline.calculate("household_weight",
                                map_to="spm_unit", period=period)

# ── Keep 4-person units that actually get SNAP (> $0) ──
mask = (unit_size == 4) & (snap > 0)

# Tiny 2-column frame for microdf
df = pd.DataFrame({"benefit": snap[mask], "wt": weights[mask]})

# Unweighted and weighted medians
median_unwtd = float(df["benefit"].median())
median_wtd   = float(mdf.weighted_median(df, "benefit", "wt"))

print(f"Median SNAP for Family of 4:   ${median_wtd:,.0f}")


4 Person Households that get Maximum SNAP Allotment :   2271.6703824460506
Median SNAP for Family of 4:   $704


In [7]:
period = "2024-01"      # keep benefits in monthly dollars

# ── Pull variables at the SPM-unit level ──
snap       = baseline.calculate("snap",          period=period)
unit_size  = baseline.calculate("spm_unit_size", period=period)
weights    = baseline.calculate("household_weight",
                                map_to="spm_unit", period=period)

# ── Keep 4-person units that actually get SNAP (> $0) ──
mask = (unit_size == 2) & (snap > 0)

# Tiny 2-column frame for microdf
df = pd.DataFrame({"benefit": snap[mask], "wt": weights[mask]})

# Unweighted and weighted medians
median_unwtd = float(df["benefit"].median())
median_wtd   = float(mdf.weighted_median(df, "benefit", "wt"))

print(f"Median SNAP for Family of 2:   ${median_wtd:,.0f}")

period = "2024-01"      # keep benefits in monthly dollars

# ── Pull variables at the SPM-unit level ──
snap       = baseline.calculate("snap",          period=period)
unit_size  = baseline.calculate("spm_unit_size", period=period)
weights    = baseline.calculate("household_weight",
                                map_to="spm_unit", period=period)

# ── Keep 4-person units that actually get SNAP (> $0) ──
mask = (unit_size == 1) & (snap > 0)

# Tiny 2-column frame for microdf
df = pd.DataFrame({"benefit": snap[mask], "wt": weights[mask]})

# Unweighted and weighted medians
median_unwtd = float(df["benefit"].median())
median_wtd   = float(mdf.weighted_median(df, "benefit", "wt"))

print(f"Median SNAP for Family of 1:   ${median_wtd:,.0f}")

Median SNAP for Family of 2:   $270
Median SNAP for Family of 1:   $114


In [8]:
# Count 4-person HOUSEHOLDS that receive any SNAP during 2024
period = 2024           # annual → sums the 12 monthly amounts

hh_size  = baseline.calculate("household_size",  map_to="household", period=period)
snap_yr  = baseline.calculate("snap",            map_to="household", period=period)
weights  = baseline.calculate("household_weight", map_to="household", period=period)

mask = (hh_size == 4) & (snap_yr > 0)            # any positive amount ⇒ got SNAP

n_unwtd = int(mask.sum())                        # simple count of CPS households
n_wtd   = float((mask * weights).sum())          # population-weighted total

print(f"Unweighted households: {n_unwtd:,}")
print(f"Weighted households:   {n_wtd:,.0f}")


Unweighted households: 583
Weighted households:   2,264,703


In [10]:
import pandas as pd
import microdf as mdf
import numpy as np

# Single month keeps it simple
period = "2024-01"

# ── Pull variables at the SPM-unit level ──
snap = baseline.calculate("snap", period=period)               # SPM-unit level
unit_size = baseline.calculate("spm_unit_size", period=period) # size of that SPM unit
weights = baseline.calculate("household_weight",
                             map_to="spm_unit", period=period) # population weights

# ── Analysis 1: Count of 4-person households getting above $965 ──
mask_above_965 = (unit_size == 4) & (snap > 965)

print("4 Person Households that get above $965 in SNAP: ", float((mask_above_965 * weights).sum()))
print()

# ── Analysis 2: Distribution statistics for these households ──
# Create dataframe for households above $965
df_above_965 = pd.DataFrame({
    "benefit": snap[mask_above_965], 
    "wt": weights[mask_above_965]
})

if len(df_above_965) > 0:
    # Calculate statistics
    median_wtd = float(mdf.weighted_median(df_above_965, "benefit", "wt"))
    mean_wtd = float((df_above_965["benefit"] * df_above_965["wt"]).sum() / df_above_965["wt"].sum())
    
    # Min and max benefits
    min_benefit = float(df_above_965["benefit"].min())
    max_benefit = float(df_above_965["benefit"].max())
    
    print(f"Statistics for 4-person households getting > $965 SNAP:")
    print(f"  Weighted median: ${median_wtd:,.0f}")
    print(f"  Weighted mean:   ${mean_wtd:,.0f}")
    print(f"  Min benefit:     ${min_benefit:,.0f}")
    print(f"  Max benefit:     ${max_benefit:,.0f}")
    print()

# ── Analysis 3: Compare to all 4-person households getting SNAP ──
mask_all_snap = (unit_size == 4) & (snap > 0)
df_all_snap = pd.DataFrame({
    "benefit": snap[mask_all_snap], 
    "wt": weights[mask_all_snap]
})

# Calculate what percentage get above $965
total_4person_snap = float((mask_all_snap * weights).sum())
pct_above_965 = (float((mask_above_965 * weights).sum()) / total_4person_snap * 100) if total_4person_snap > 0 else 0

print(f"Of all 4-person households getting SNAP:")
print(f"  Total households: {total_4person_snap:,.0f}")
print(f"  Getting > $965:   {float((mask_above_965 * weights).sum()):,.0f} ({pct_above_965:.1f}%)")

# ── Analysis 4: Look at specific benefit thresholds ──
print("\nBenefit distribution for 4-person households:")
thresholds = [500, 750, 965, 1000, 1200]
for threshold in thresholds:
    mask_threshold = (unit_size == 4) & (snap >= threshold)
    count = float((mask_threshold * weights).sum())
    pct = (count / total_4person_snap * 100) if total_4person_snap > 0 else 0
    print(f"  >= ${threshold:4d}: {count:10,.0f} ({pct:5.1f}%)")

4 Person Households that get above $965 in SNAP:  51343.36664519112

Statistics for 4-person households getting > $965 SNAP:
  Weighted median: $973
  Weighted mean:   $973
  Min benefit:     $973
  Max benefit:     $1,456

Of all 4-person households getting SNAP:
  Total households: 175,704
  Getting > $965:   51,343 (29.2%)

Benefit distribution for 4-person households:
  >= $ 500:    127,950 ( 72.8%)
  >= $ 750:     79,433 ( 45.2%)
  >= $ 965:     51,343 ( 29.2%)
  >= $1000:          0 (  0.0%)
  >= $1200:          0 (  0.0%)
